In [1]:
import pickle
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
path = "/kaggle/input/dataset-chest/"

In [3]:
df_chest_filt = pd.read_pickle((path + "combined_chest_filtered.pkl"))

In [4]:
df_chest_filt.head()

,id,ACC_x,ACC_y,ACC_z,ECG,EMG,EDA,TEMP,RESP,label
214583,2.0,0.8914,-0.1102,-0.2576,0.030945,-0.003708,5.710983,29.083618,1.191711,1.0
214584,2.0,0.8926,-0.1086,-0.2544,0.033646,-0.014145,5.719376,29.122437,1.139832,1.0
214585,2.0,0.8930,-0.1094,-0.2580,0.033005,0.010208,5.706406,29.115234,1.141357,1.0
214586,2.0,0.8934,-0.1082,-0.2538,0.031815,0.012634,5.712509,29.126709,1.155090,1.0
214587,2.0,0.8930,-0.1096,-0.2570,0.030350,0.002060,5.727005,29.100861,1.133728,1.0


In [5]:
df_chest_filt.isnull().sum()

id       0
ACC_x    0
ACC_y    0
ACC_z    0
ECG      0
EMG      0
EDA      0
TEMP     0
RESP     0
label    0
dtype: int64

In [18]:
df_chest_filt.isna().any()

id       False
ACC_x    False
ACC_y    False
ACC_z    False
ECG      False
EMG      False
EDA      False
TEMP     False
RESP     False
label    False
dtype: bool

In [17]:
# Extraer features y labels
X = df_chest_filt[['ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'EDA', 'TEMP', 'RESP']].values
y = df_chest_filt['label'].values

# Clasificación binaria para los datos de pecho

In [24]:
# Reducción de dimensiones con el método PCA para facilitar la ejecución del modelo
pca = PCA(n_components=0.95)
reduced_X_PCA = pca.fit_transform(X)
print("Dimensiones reducidas con PCA:", np.shape(reduced_X_PCA))

Dimensiones reducidas con PCA: (23206321, 3)


In [9]:
# Clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_binary = np.where((y == 1) | (y == 3), 0, 1)
# Separar los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(reduced_X_PCA, y_binary, test_size=0.2, random_state=50)

In [26]:
# Crear y entrenar el clasificador Random Forest
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=50)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=50)

In [27]:
# Predicción
y_pred = rf_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Precisión: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Precisión: 0.9875
Recall: 0.9730
F1 Score: 0.9791
Confusion Matrix:
[[3225898   20396]
 [  37689 1357282]]


In [7]:
# Modifica los parametros para intentar mejorar los resultados:
# PCA con n_componentes=0,99
pca = PCA(n_components=0.99)
reduced_X_PCA = pca.fit_transform(X)
print("Dimensiones reducidas con PCA:", np.shape(reduced_X_PCA))

Dimensiones reducidas con PCA: (23206321, 3)


In [11]:
# Random Forest n_estimators=20
X_train, X_test, y_train, y_test = train_test_split(reduced_X_PCA, y_binary, test_size=0.2, random_state=50)

# Crear y entrenar el clasificador Random Forest
rf_classifier = RandomForestClassifier(n_estimators=20, random_state=50)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20, random_state=50)

In [12]:
# Predicción
y_pred = rf_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Precisión: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Precisión: 0.9880
Recall: 0.9751
F1 Score: 0.9799
Confusion Matrix:
[[3225360   20934]
 [  34781 1360190]]


No hay mejoras significantes y ha tardado más para ejecutar. La precisión conseguido con n_componentes=0.95 y n_estimators=10 ya están bastante bien. Por lo tanto, se puede utilizar estos valores para la clasificaión de tres estados afectivos.

# Clasificación de tres estados afectivos para los datos de pecho

In [6]:
# Vuelve con PCA n=0,95 para la clasificación de tres estados afectivos
pca = PCA(n_components=0.95)
reduced_X_PCA = pca.fit_transform(X)
print("Dimensiones reducidas con PCA:", np.shape(reduced_X_PCA))

Dimensiones reducidas con PCA: (23206321, 3)


In [18]:
# One hot encoding para tres clases
y = y - 1
y_3status = to_categorical(y, num_classes=3)

# Separar los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(reduced_X_PCA, y_3status, test_size=0.2, random_state=50)


In [19]:
y_3status

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [21]:
# Crear y entrenar el clasificador Random Forest
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=50)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=50)

In [25]:
# Predicción
# Convertir los vectores transformados por one hot encoding a formato original para 
# poder aplicar las metricas
y_pred = np.argmax(rf_classifier.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)

print(f"Precisión: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Precisión: 0.9792
Recall: 0.9792
F1 Score: 0.9791
Confusion Matrix:
[[2441350    9026   14598]
 [  24556 1357459   12956]
 [  24200   11392  745728]]
